In [1]:
from pythainlp.tokenize import word_tokenize
from gensim.models import KeyedVectors
import numpy as np
import re
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import string
import pandas as pd
model_path = 'thwiki_data/models/'

In [2]:
from pythainlp import word_vector
model = word_vector.get_model()

## Setting

In [3]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Bidirectional
import nltk
from tensorflow.keras.layers import *
def model(X,Y):
  m=Sequential()
  m.add(Bidirectional(LSTM(256, return_sequences=True, input_shape=(1,300))))
  m.add(Bidirectional(LSTM(128, return_sequences=True, input_shape=(1,300))))
  # m.add(Bidirectional(LSTM(64, return_sequences=True, input_shape=(1,300))))
  # m.add(Bidirectional(LSTM(32, return_sequences=True, input_shape=(1,300))))
  m.add(Dense(8, activation="sigmoid"))
  m.compile(loss='binary_crossentropy', optimizer='adam')
  return m

In [4]:
df_data = pd.read_csv('../Data/THR_DLR_CLA_V2_Primary_Full_EA_New_Format_Shuffle.csv')

In [5]:
def clean_msg(msg):
    # ลบ text ที่อยู่ในวงเล็บ <> ทั้งหมด
    msg = re.sub(r'<.*?>','', msg)
    # ลบ hashtag
    msg = re.sub(r'#','',msg)
    # ลบ เครื่องหมายคำพูด (punctuation)
    for c in string.punctuation:
        msg = re.sub(r'\{}'.format(c),'',msg)
    # ลบ separator เช่น \n \t
    msg = ' '.join(msg.split())
    return msg

In [6]:
X = df_data.iloc[:,3:4].values 
y = df_data.iloc[:, 5:].values
clean_text_arr = []
for i in range(len(X)):
    clean_text = clean_msg(X[i][0])
    temp_arr = []
    temp_arr.append(clean_text)
    clean_text_arr.append(temp_arr)
# print(clean_text_arr)
X = np.array(clean_text_arr)
print(len(y[0]))

8


In [7]:
Tokenized_Sentence = []
for sentence in X:
    Tokenized_Sentence.append(word_vector.sentence_vectorizer(sentence[0]).tolist())

In [8]:
Rearranged_X = []
for data in Tokenized_Sentence:
    Rearranged_X.append(data[0])

In [9]:
def MCC(TP,TN,FP,FN):
    return ((TP*TN - FP * FN) / math.sqrt((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN)))

def average(list):
    return sum(list) / len(list)

def accuracy(TP,TN,FP,FN):
    return ((TP + TN) / (TP + TN + FP + FN))

In [10]:
len(Rearranged_X[0])

300

In [11]:
Rearranged_X = np.array(Rearranged_X)

In [12]:
temp_pd = pd.DataFrame(columns=['Fold','Category','Precision','Recall','F1-Score','Support','Accuracy','MCC','Roc_Auc_Cruve'])

In [19]:
from skmultilearn.model_selection import iterative_train_test_split
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix, multilabel_confusion_matrix, roc_auc_score
from sklearn.model_selection import KFold
import math
import pickle
Result_arr = [[],[],[],[],[],[],[],[]]
ROC_Result_arr = [[],[],[],[],[],[],[],[]]
Confusion_matrix_arr = [[],[],[],[],[],[],[],[]]
Classification_report_list = []
Class = ['0','1','2','3','4','5','6','7','macro avg']
Sub_Class_list = ['precision','recall','f1-score','support']
news_label = ['Gambling', 'Murder', 'Sexual_Abuse', 'Theft/Burglary', 'Drug', 'Battery/Assault', 'Accident', 'Non-Crime', 'Macro Avg']
kf = KFold(n_splits=10)
kf.get_n_splits(Rearranged_X)
Fold = 1
for train_index, test_index in kf.split(Rearranged_X):
    X_train, X_test, Y_train, Y_test = Rearranged_X[train_index], Rearranged_X[test_index], y[train_index], y[test_index]
    X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.1111)
    print(X_train.shape)
    print(Y_train.shape)
    print(X_val.shape)
    print(Y_val.shape)
    X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
    X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))
    X_val = np.reshape(X_val, (X_val.shape[0], 1, X_val.shape[1]))
    Y_train = np.reshape(Y_train, (Y_train.shape[0], 1, Y_train.shape[1]))
    Y_test = np.reshape(Y_test, (Y_test.shape[0], 1, Y_test.shape[1]))
    Y_val = np.reshape(Y_val, (Y_val.shape[0], 1, Y_val.shape[1]))
    bilstmModel=model(X_train,Y_train)
    bilstmModel.fit(X_train,Y_train,epochs=100, batch_size=128,validation_data=(X_val,Y_val))
    result = bilstmModel.predict(X_test)
    result = np.round(result)
    result = result.reshape(result.shape[0],result.shape[2])
    file = open(f'../Data/Predicted_pickle/{Fold}_Predicted_BiLSTM.pkl','wb')
    pickle.dump(result,file) 
    Y_test = Y_test.reshape(Y_test.shape[0],Y_test.shape[2])
    MCM_array = multilabel_confusion_matrix(Y_test,result)
    class_rep = classification_report(Y_test,result,output_dict=True)
    for category in range(0,8,1):
        print(MCM_array[category])
        print(category)
        Result_arr[category].append(MCC(MCM_array[category][0][0],MCM_array[category][1][1],MCM_array[category][1][0],MCM_array[category][0][1]))
        auc = roc_auc_score(Y_test[:,category],result[:,category])
        ROC_Result_arr[category].append(auc)
        Confusion_matrix_arr[category].append(MCM_array[category])
        precision = class_rep[Class[category]][Sub_Class_list[0]]
        recall = class_rep[Class[category]][Sub_Class_list[1]]
        f1_score = class_rep[Class[category]][Sub_Class_list[2]]
        support = class_rep[Class[category]][Sub_Class_list[3]]
        accuracy_result = accuracy(MCM_array[category][0][0],MCM_array[category][1][1],MCM_array[category][1][0],MCM_array[category][0][1])
        MCC_result = MCC(MCM_array[category][0][0],MCM_array[category][1][1],MCM_array[category][1][0],MCM_array[category][0][1])
        temp_row = pd.Series([Fold,news_label[category],precision,recall,f1_score,support,accuracy_result,MCC_result,auc],index=temp_pd.columns)
        temp_pd = temp_pd.append(temp_row,ignore_index=True)
    temp_row = pd.Series([Fold,news_label[8],class_rep[Class[8]][Sub_Class_list[0]],class_rep[Class[8]][Sub_Class_list[1]],class_rep[Class[8]][Sub_Class_list[2]],class_rep[Class[8]][Sub_Class_list[3]],0,0,0],index=temp_pd.columns)
    temp_pd = temp_pd.append(temp_row,ignore_index=True)
    Classification_report_list.append(class_rep)
    Fold += 1

(6850, 300)
(6850, 8)
(857, 300)
(857, 8)
Epoch 1/100
54/54 [==============================] - 3s 17ms/step - loss: 0.4403 - val_loss: 0.3571
Epoch 2/100
54/54 [==============================] - 0s 7ms/step - loss: 0.3667 - val_loss: 0.3440
Epoch 3/100
54/54 [==============================] - 0s 7ms/step - loss: 0.3349 - val_loss: 0.3036
Epoch 4/100
54/54 [==============================] - 0s 6ms/step - loss: 0.3104 - val_loss: 0.2903
Epoch 5/100
54/54 [==============================] - 0s 7ms/step - loss: 0.2914 - val_loss: 0.2685
Epoch 6/100
54/54 [==============================] - 0s 7ms/step - loss: 0.2654 - val_loss: 0.2445
Epoch 7/100
54/54 [==============================] - 0s 7ms/step - loss: 0.2474 - val_loss: 0.2353
Epoch 8/100
54/54 [==============================] - 0s 6ms/step - loss: 0.2343 - val_loss: 0.2210
Epoch 9/100
54/54 [==============================] - 0s 7ms/step - loss: 0.2234 - val_loss: 0.2120
Epoch 10/100
54/54 [==============================] - 0s 7ms/step 

/home/crimson/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/crimson/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


54/54 [==============================] - 5s 16ms/step - loss: 0.4393 - val_loss: 0.3674
Epoch 2/100
54/54 [==============================] - 0s 6ms/step - loss: 0.3664 - val_loss: 0.3592
Epoch 3/100
54/54 [==============================] - 0s 6ms/step - loss: 0.3386 - val_loss: 0.3205
Epoch 4/100
54/54 [==============================] - 0s 7ms/step - loss: 0.3117 - val_loss: 0.3042
Epoch 5/100
54/54 [==============================] - 0s 7ms/step - loss: 0.2959 - val_loss: 0.2865
Epoch 6/100
54/54 [==============================] - 0s 7ms/step - loss: 0.2742 - val_loss: 0.2677
Epoch 7/100
54/54 [==============================] - 0s 7ms/step - loss: 0.2543 - val_loss: 0.2550
Epoch 8/100
54/54 [==============================] - 0s 6ms/step - loss: 0.2398 - val_loss: 0.2380
Epoch 9/100
54/54 [==============================] - 0s 7ms/step - loss: 0.2264 - val_loss: 0.2358
Epoch 10/100
54/54 [==============================] - 0s 7ms/step - loss: 0.2180 - val_loss: 0.2231
Epoch 11/100
54/54 [

/home/crimson/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/crimson/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


54/54 [==============================] - 4s 27ms/step - loss: 0.4393 - val_loss: 0.3693
Epoch 2/100
54/54 [==============================] - 0s 7ms/step - loss: 0.3657 - val_loss: 0.3545
Epoch 3/100
54/54 [==============================] - 0s 7ms/step - loss: 0.3332 - val_loss: 0.3139
Epoch 4/100
54/54 [==============================] - 0s 7ms/step - loss: 0.3097 - val_loss: 0.2979
Epoch 5/100
54/54 [==============================] - 0s 7ms/step - loss: 0.2893 - val_loss: 0.2705
Epoch 6/100
54/54 [==============================] - 0s 7ms/step - loss: 0.2624 - val_loss: 0.2465
Epoch 7/100
54/54 [==============================] - 0s 6ms/step - loss: 0.2452 - val_loss: 0.2322
Epoch 8/100
54/54 [==============================] - 0s 7ms/step - loss: 0.2315 - val_loss: 0.2250
Epoch 9/100
54/54 [==============================] - 0s 7ms/step - loss: 0.2204 - val_loss: 0.2130
Epoch 10/100
54/54 [==============================] - 0s 6ms/step - loss: 0.2112 - val_loss: 0.2142
Epoch 11/100
54/54 [

...

/home/crimson/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/crimson/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


54/54 [==============================] - 4s 16ms/step - loss: 0.4407 - val_loss: 0.3667
Epoch 2/100
54/54 [==============================] - 0s 7ms/step - loss: 0.3663 - val_loss: 0.3544
Epoch 3/100
54/54 [==============================] - 0s 7ms/step - loss: 0.3366 - val_loss: 0.3137
Epoch 4/100
54/54 [==============================] - 0s 6ms/step - loss: 0.3114 - val_loss: 0.2982
Epoch 5/100
54/54 [==============================] - 0s 6ms/step - loss: 0.2924 - val_loss: 0.2773
Epoch 6/100
54/54 [==============================] - 0s 7ms/step - loss: 0.2652 - val_loss: 0.2514
Epoch 7/100
54/54 [==============================] - 0s 7ms/step - loss: 0.2462 - val_loss: 0.2404
Epoch 8/100
54/54 [==============================] - 0s 7ms/step - loss: 0.2326 - val_loss: 0.2250
Epoch 9/100
54/54 [==============================] - 0s 6ms/step - loss: 0.2217 - val_loss: 0.2203
Epoch 10/100
54/54 [==============================] - 0s 7ms/step - loss: 0.2147 - val_loss: 0.2118
Epoch 11/100
54/54 [

/home/crimson/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/crimson/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


54/54 [==============================] - 4s 17ms/step - loss: 0.4434 - val_loss: 0.3759
Epoch 2/100
54/54 [==============================] - 0s 7ms/step - loss: 0.3656 - val_loss: 0.3626
Epoch 3/100
54/54 [==============================] - 0s 7ms/step - loss: 0.3342 - val_loss: 0.3240
Epoch 4/100
54/54 [==============================] - 0s 7ms/step - loss: 0.3078 - val_loss: 0.3048
Epoch 5/100
54/54 [==============================] - 0s 7ms/step - loss: 0.2906 - val_loss: 0.2823
Epoch 6/100
54/54 [==============================] - 0s 7ms/step - loss: 0.2634 - val_loss: 0.2569
Epoch 7/100
54/54 [==============================] - 0s 7ms/step - loss: 0.2434 - val_loss: 0.2400
Epoch 8/100
54/54 [==============================] - 0s 6ms/step - loss: 0.2301 - val_loss: 0.2284
Epoch 9/100
54/54 [==============================] - 0s 7ms/step - loss: 0.2203 - val_loss: 0.2199
Epoch 10/100
54/54 [==============================] - 0s 6ms/step - loss: 0.2124 - val_loss: 0.2130
Epoch 11/100
54/54 [

/home/crimson/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/crimson/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [20]:
temp_pd

,Fold,Category,Precision,Recall,F1-Score,Support,Accuracy,MCC,Roc_Auc_Cruve
0,1,Gambling,0.857143,0.600000,0.705882,20,0.988331,0.711735,0.798805
1,1,Murder,0.794521,0.849817,0.821239,273,0.882147,0.734408,0.873539
2,1,Sexual_Abuse,0.651515,0.741379,0.693548,58,0.955659,0.671381,0.856297
3,1,Theft/Burglary,0.739130,0.531250,0.618182,64,0.950992,0.601985,0.758059
4,1,Drug,0.716814,0.764151,0.739726,106,0.933489,0.702115,0.860771
...,...,...,...,...,...,...,...,...,...
184,10,Drug,0.718447,0.732673,0.725490,101,0.934579,0.688402,0.847131
185,10,Battery/Assault,0.710526,0.437838,0.541806,185,0.839953,0.470825,0.694329
186,10,Accident,0.788462,0.602941,0.683333,68,0.955607,0.666791,0.794491
187,10,Non-Crime,0.783784,0.664122,0.719008,131,0.920561,0.676250,0.815509


In [21]:
def average_acc(index):
    TP_list = []
    FP_list = []
    FN_list = []
    TN_list = []
    TP_list.append(Confusion_matrix_arr[index][0][0][0])
    FP_list.append(Confusion_matrix_arr[index][0][0][1])
    FN_list.append(Confusion_matrix_arr[index][0][1][0])
    TN_list.append(Confusion_matrix_arr[index][0][1][1])
    return ((sum(TP_list) + sum(TN_list))/(sum(TP_list) + sum(FP_list) + sum(FN_list) + sum(TN_list)))

In [22]:
Class = ['0','1','2','3','4','5','6','7','macro avg']
Sub_Class_list = ['precision','recall','f1-score','support']
Classification_result = [[[],[],[],[]],[[],[],[],[]],[[],[],[],[]],[[],[],[],[]],[[],[],[],[]],[[],[],[],[]],[[],[],[],[]],[[],[],[],[]],[[],[],[],[]]]
for a in range(0,10,1):
    for i in range(0,9,1):
        for j in range(0,4,1):
            Classification_result[i][j].append(Classification_report_list[a][Class[i]][Sub_Class_list[j]])
for i in range(0,9,1):
    print(f'{news_label[i]}: {average(Classification_result[i][0])} {average(Classification_result[i][1])} {average(Classification_result[i][2])} {average(Classification_result[i][3])}')
        

Gambling: 0.8036598214603445 0.6391443980477993 0.702880202719151 24.9
Murder: 0.8478583142239489 0.8156635844383724 0.8293812331216456 255.5
Sexual_Abuse: 0.7310747369910948 0.6682255084265571 0.6930984149004449 67.3
Theft/Burglary: 0.7168687568430536 0.5528885899150737 0.6215270310762386 77.4
Drug: 0.7840917899571248 0.726915800796389 0.7533697945878413 103.9
Battery/Assault: 0.6905663561391374 0.5310582938877565 0.5974263050113002 188.8
Accident: 0.7941061629510785 0.7001760046089747 0.7425619118680824 72.1
Non-Crime: 0.8184050195710316 0.7028111929794814 0.755203927128588 140.6
Macro Avg: 0.7733288697671018 0.6671104216375505 0.7119311025516615 930.5


In [23]:
for category in range(0,8,1):
    avg_precision = average(Classification_result[category][0])
    avg_recall = average(Classification_result[category][1])
    avg_f1 = average(Classification_result[category][2])
    avg_support = average(Classification_result[category][3])
    avg_accuracy = average_acc(category)
    avg_MCC = average(Result_arr[category])
    avg_roc = average(ROC_Result_arr[category])
    temp_row = pd.Series([-1,news_label[category],avg_precision,avg_recall,avg_f1,avg_support,avg_accuracy,avg_MCC,avg_roc],index=temp_pd.columns)
    temp_pd = temp_pd.append(temp_row,ignore_index=True)
temp_row = pd.Series([-1,news_label[8],average(Classification_result[i][0]),average(Classification_result[i][1]),average(Classification_result[i][2]),average(Classification_result[i][3]),0,0,0],index=temp_pd.columns)
temp_pd = temp_pd.append(temp_row,ignore_index=True)

In [24]:
temp_pd.to_excel('./K_Fold_Metric_Evaluation.xlsx',index=False)